#### Bi 410/510 (Fall 2019)

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

**Group Members**

If this is a group submission edit this cell to add the names and e-mail addresses of the othrer people who worked on the project.
* name (xxx@uoregon.edu)
* name (xxx@uoregon.edu)
* name (xxx@uoregon.edu)

# <span style="color:seagreen;">Project 5: &nbsp; I/O and Dictionaries</span>

###  <span style="color:seagreen">Instructions</span>

In previous projects we wrote the `def` statements of functions for you, which meant the code cells you need to fill in looked something like this:
```
def foo():
    pass
```

Starting with this project there will be cells where you have to write the `def` statement yourself.  The code cell will simply have the word `pass`.  Delete that line and replace it with the complete definition of your function.

###  <span style="color:seagreen">Data Files</span>

To complete this project you will need to download two files from the Bi410 server.
The easiest way to download the files is to start a Docker shell, `cd` to the directory where you saved this notebook, and use the `download` shortcut:
```
$ download clusters.txt
$ download example_dictionaries.py
```

This project uses two definitions from `example_dictionaries.py`, one called `gc` ("genetic code") and one called `mol_wt` ("molecular weight")
* because `example_dictionaries.py` is in the same directory as this notebook you can use an `import` statement to load a dictionary into this notebook
* we've provided code cells with `import` statements

**NOTE:** if the two files are not saved in the same directory as this notebook you will get an error message when you try to import a dictionary or open the data file.

###  <span style="color:seagreen">Exercise 5.1: Clusters</span> 

The file named `clusters.txt` is a TSV (tab separated values) file produced by a program that analyzes DNA sequences and organizes them into clusters of similar sequences. The relevant information for this project is in the first two columns of each line:
* Column 1 has a sequence ID, which is a long string that starts with a series of numbers and ends with ’size=’ (you’ll use the size in the next project, but ignore it for now).
* Column 2 has a string, either ’otu’ or ’match’.

Implement a function named `cluster_ids` that reads a cluster description file and returns a list of the sequence ID fields for every OTU, _i.e._ for every record that has `otu` in the second column. 

This is what the first two lines should look like for this data set:
```
>>> cluster_ids('clusters.txt')
['715:51:674:768;id=2007;size=36',
 '810:412:436:962;id=119;size=14',
 ...
]
```

**Hint:** Since the fields on each line are separated by tabs, you can use something like this to split a line into a list of fields:
```
   line.split('\t')
```

In [1]:
def cluster_ids(fn):
    lst = []
    for line in open('clusters.txt'):
        data_line = line.strip().split('	')
        if 'otu' in data_line:
            lst.append(data_line[0])
     
    return(lst)

##### Tests 

Use this code cell to test your function as you work on it:

In [2]:
cluster_ids('clusters.txt')

['715:51:674:768;id=2007;size=36',
 '810:412:436:962;id=119;size=14',
 '259:174:253:24;id=2281;size=11',
 '3080:32:2633:30;id=1940;size=10',
 '1616:325:49:202;id=295;size=5',
 '93:74:93:30;id=2400;size=5',
 '4227:1088:2615:232;id=2137;size=4']

We'll use this code cell to verify the code works:

In [3]:
assert len(cluster_ids('clusters.txt')) == 7

###  <span style="color:seagreen">Exercise 5.2: Cluster Statistics</span>

Implement a function named `cluster_stats` that will read a cluster description file and print some statistics about cluster sizes.

**Hint:** see the lecture notes on "ad hoc parsing" (in `IO II.ipynb`) for examples of code that extracts small substrings from a longer data line.

The program should print the number of clusters, the total number of sequences in all clusters, and the mean cluster size. 

For example, suppose there are three clusters in a file:
```
30:321:33:30;id=940;size=10  otu   74.8  * 715:51:74:768;id=20
61:325:49:19;id=295;size=7   otu   83.0  * 715:51:74:768;id=20
93:074:93:31;id=200;size=5   otu   85.4  * 715:51:74:768;id=20
```
The number of clusters is simply the number of lines in the file.
The number of sequences in a cluster is the number to the right of `size=` in the first field.
In this example the sizes are 10, 7, and 5, so the total number of sequences is 22 and the mean cluster size is 22 / 3 = 7.33.

The test file named `clusters.txt` has 11 clusters, with a total of 99 sequences in all clusters, so this is what you should see when you process that file with your function:
```
>>> cluster_stats('clusters.txt')
(11, 99, 9.0)
```

In [4]:
def cluster_stats(fn):
    row_num = []
    for row in open(fn):
        size = row.split('=')[2].split("\t")
        
        clus = (size[0])
        
        row_num.append(int(clus))
   
    total_seq = len(row_num)
    mean = sum(row_num)/total_seq
    
    return (total_seq, sum(row_num), mean)

##### Tests 

Use this code cell to test your code.

In [5]:
cluster_stats('clusters.txt')

(11, 99, 9.0)

We'll use this cell to test your function:

In [6]:
assert cluster_stats('clusters.txt') == (11, 99, 9.0)

###  <span style="color:seagreen">Exercise 5.3: Translate</span>

Define a function named `translate` that will translate a DNA sequence into an amino acid sequence.

For the previous project you wrote a program that iterated over a DNA string and printed each codon. In this project you want to look up each codon in the dictionary named `gc` to find the corresponding amino acid letter and then append that letter to your output string.

Here is an example with a valid input string, where all letters are in the “DNA alphabet” and the length of the input is a multiple of 3:
```
>>> translate('GATTACATG')
DYM
```

If one of the codons has a letter other than `A`, `C`, `G`, or `T` the codon will not be in the dictionary.  In that case add a question mark to the output string.
The same is true if the length of the input is not a multiple of 3 -- there will be a partial codon left over, and you should add a question mark.  The result of this call has question marks for both cases:
```
>>> translate('GATXAAATGA')
D?M?
```

In [7]:
from example_dictionaries import gc

def translate(seq):
    ls = []
    tl = []

    for i in range(len(seq)//3):   
            codon = seq[i*3:i*3+3]
            ls.append(codon)
    
    for ls_codon in ls:
        if ls_codon in gc.keys():
            tl.append(gc[ls_codon])
        else:
            tl.append('?')
            
        
    if len(seq) % 3 != 0:
        tl.append('?')
        
    return ''.join(tl)
    
            
#     if len(seq) % 3 != 0:
#         ls.append('?')
    
        
#         translation = ''.join(ls)
#         print(translation)
        
    
    
    
# WRONG THING BUT THIS THING WAS COOL:
#     for ch in seq:
#         if ch in gc.values():
#             ls.append(ch)
#         if ch not in gc.values():
#             ls.append('?')
            
#     if len(seq)% 3 != 0:
#         ls.append('?')
        
#     translation = ''.join(ls)
#     return translation

##### Tests 

Use this code cell to test your function as you work on it.

In [8]:
from example_dictionaries import gc

In [9]:
gc['TTT']

'F'

In [10]:
a ='GATTACATGWB'

In [11]:
print('?')

?


In [12]:
blah = []

In [13]:
blah.append('hello')

In [14]:
len('GATXAAATG') % 3

0

In [15]:
translate('AAACCGTAG')

'KP*'

In [16]:
translate('GATXAAATGA')

'D?M?'

In [17]:
translate('GATTACATG')

'DYM'

In [18]:
translate(a)

'DYM?'

We'll use these code cells to test your function:

In [19]:
assert translate('GATTACATG') == 'DYM'

In [20]:
assert translate('GATXAAATGA') == 'D?M?'

###  <span style="color:seagreen">Exercise 5.4: Molecular Weight</span>

Define a function named `weight` that will compute the molecular weight of a protein.  The molecular weight of a protein is the sum of the weights of each amino acid. 
* the molecular weights of amino acids are in a dictionary named `mol_wt` (one of the dictionaries in `example_dictionaries.py`)

If a character in the input sequence is not an amino acid letter return `None`.

Example:  the molecular weight of `MVL` should be 149.21 + 117.15 + 131.17 = 397.53
```
>>> weight('MVL')
397.53

>>> weight('MXVL')
warning: unknown letter: X
397.53
```

**Note:** This project is one of the examples in the textbook.  See if you can solve it by yourself first, but if you get stuck you can look at their code for inspiration.

In [21]:
from example_dictionaries import mol_wt

def weight(aas):
    mol_wt_lst = []
    for aa in aas:
        if aa not in mol_wt.keys():
            return None
        else:
            mol_wt_lst.append(mol_wt[aa])
    tot_mw = sum(mol_wt_lst)
    return tot_mw
            

##### Tests 

Use this code cell to test your function as you work on it.

In [22]:
mol_wt["A"]

89.09

In [23]:
l = [mol_wt["A"], mol_wt["C"]]

In [24]:
sum(l)

210.24

In [25]:
weight('MVL')

397.53

We'll use these code cells to test your function.

In [26]:
assert weight('MVL') == 397.53

In [27]:
assert weight('MXVL') is None